In [6]:
import numpy as np
import pandas as pd
import string
import re

#sklearn imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

#scipy imports
from scipy.sparse import hstack

#Visualization imports
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import bokeh
#! pip install bokeh

# target classes
target_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [210]:
# read frames localy through csv
train_df = pd.read_csv("../../data/train.csv")
test_df = pd.read_csv("../../data/test.csv")
train_data = train_df["comment_text"]

# starting from the /usr/share/dict/words set of half a million words with abbreviations
# and others, and adding to them any obvious missing values
filename = 'words.txt'
file = open(filename, 'rt')
all_words = file.read()
file.close
all_words = all_words.split()
all_words = [ word.lower() for word in all_words]
all_words = set(all_words)

from nltk.corpus import brown
good_words_list = brown.words()
good_word_set = set([word.lower() for word in good_words_list])
#punctuation = re.sub("[\'\-]",'',string.punctuation)
punctuation = "[\!\?\"\#\$\%\&\(\)\*\+\,\.\/\:\;\<\=\>\?\@\[\]\^\_\`\{\|\}\~\']"

done


In [248]:
from autocorrect import spell

def find_spelling_errors(textdoc):
    words = textdoc.split()
    words = [ word.lower() for word in words]
    misspelled_words = []
    misspelled_words.append([re.sub(punctuation,'',word) for word in words if re.sub(punctuation,'',word) not in all_words])
    return misspelled_words

def trysplit(word):
    lentmp = len(word)
    return_words=[]
    for i in range(lentmp):
        if all_words.intersection(set([word[:lentmp-i].lower()])):
            return_words.append(word[:lentmp-i])
            if len(word[lentmp-i:]) > 0:
                return_words.extend(trysplit(word[lentmp-i:]))
                print(return_words)
            else:
                return [word[:lentmp-i]]
            break
    return return_words 

def fix_spelling_errors(textdoc):
    words = textdoc.split()
    return_list = []
    for word in words:
        
        if all_words.intersection(set([re.sub(punctuation,'',word.lower())])):
            return_list.extend(word)
        else:
            # word is not found in the dictionary, try to correct the spelling
            if word == spell(word): # no changes made by the spell checker
                return_list.extend(trysplit(word))
            else:
                return_list.extend(spell(word))
    return return_list
    #return [ spell(word.lower()) if re.sub(punctuation,'',word.lower()) not in all_words else word for word in words]
    #return [spell(word) for word in words]

In [249]:
# find all the spelling errors
spelling_errors=[]
for index in range(1):
    spelling_errors.append(find_spelling_errors(train_data[index]))

flat_list = []
for sublist in spelling_errors:
    for sublist2 in sublist:
        for item in sublist2:
            flat_list.append(item)
            
# y = np.bincount(np.unique(flat_list))
# ii = np.nonzero(y)[0]
# out = np.vstack((ii, y[ii])).T
import collections
collections.Counter(flat_list)
# print(np.unique(flat_list, return_counts=True))
# print(flat_list)
x=8
print(train_data[x])
print('Invalid spellings: %s' % (find_spelling_errors(train_data[x])))
print(fix_spelling_errors(train_data[x]))



Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169
Invalid spellings: [['articlewow', '93161107169']]
['article']
['S', 'o', 'r', 'r', 'y', 'i', 'f', 't', 'h', 'e', 'w', 'o', 'r', 'd', "'", 'n', 'o', 'n', 's', 'e', 'n', 's', 'e', "'", 'w', 'a', 's', 'o', 'f', 'f', 'e', 'n', 's', 'i', 'v', 'e', 't', 'o', 'y', 'o', 'u', '.', 'A', 'n', 'y', 'w', 'a', 'y', ',', 'I', "'", 'm', 'n', 'o', 't', 'i', 'n', 't', 'e', 'n', 'd', 'i', 'n', 'g', 't', 'o', 'w', 'r', 'i', 't', 'e', 'a', 'n', 'y', 't', 'h', 'i', 'n', 'g', 'i', 'n', 't', 'h', 'e', 'article', 't', 'h', 'e', 'y', 'w', 'o', 'u

In [237]:
def trysplit_orig(word):
    lentmp = len(word)
    return_words=[]
    print("word incoming is :" + word)
    for i in range(lentmp):
        print("Checking " + word[:lentmp-i])
        if all_words.intersection(set([word[:lentmp-i].lower()])):
            print("Found word:" + word[:lentmp-i] )
            return_words.append(word[:lentmp-i])
            if len(word[lentmp-i:]) > 0:
                return_words.extend(trysplit(word[lentmp-i:]))
                print("Current return words are: ")
                print(return_words)
            else:
                print("Returning last word " + word[:lentmp-i])
                return [word[:lentmp-i]]
            break
    return return_words   
    
print(trysplit('ajdlfjaljsdf'))


word incoming is :ajdlfjaljsdf
Checking ajdlfjaljsdf
Checking ajdlfjaljsd
Checking ajdlfjaljs
Checking ajdlfjalj
Checking ajdlfjal
Checking ajdlfja
Checking ajdlfj
Checking ajdlf
Checking ajdl
Checking ajd
Checking aj
Found word:aj
word incoming is :dlfjaljsdf
Checking dlfjaljsdf
Checking dlfjaljsd
Checking dlfjaljs
Checking dlfjalj
Checking dlfjal
Checking dlfja
Checking dlfj
Checking dlf
Checking dl
Found word:dl
word incoming is :fjaljsdf
Checking fjaljsdf
Checking fjaljsd
Checking fjaljs
Checking fjalj
Checking fjal
Checking fja
Checking fj
Checking f
Found word:f
word incoming is :jaljsdf
Checking jaljsdf
Checking jaljsd
Checking jaljs
Checking jalj
Checking jal
Found word:jal
word incoming is :jsdf
Checking jsdf
Checking jsd
Found word:jsd
word incoming is :f
Checking f
Found word:f
Returning last word f
Current return words are: 
['jsd', 'f']
Current return words are: 
['jal', 'jsd', 'f']
Current return words are: 
['f', 'jal', 'jsd', 'f']
Current return words are: 
['dl', 'f', 

In [224]:
myword="fuck"
if all_words.intersection(set([myword])):
    print(all_words.intersection(set([myword])))
#print(all_words.shape)

{'fuck'}


In [236]:
x="the"
if x == spell(x):
    print("no match for " + x)
    
spell("I'm")

no match for the


'Ism'